# Phase 10: Issue Discovery Demo

This notebook demonstrates passive issue discovery without the full UI.

In [ ]:
import sys

sys.path.insert(0, "d:/odibi")

from agents.core.issue_discovery import (
    IssueDiscoveryManager,
    PassiveIssueDetector,
)

## 1. Run Passive Issue Discovery

Scan the examples directory for potential issues.

In [ ]:
# Initialize the discovery manager
manager = IssueDiscoveryManager(odibi_root="d:/odibi")

# Scan the examples directory
result = manager.run_discovery(project_root="d:/odibi/examples", cycle_id="demo-scan-001")

print(f"Scanned {len(result.files_scanned)} file(s)")
print(f"Found {result.total_issues} issue(s)")
print(f"Actionable: {result.actionable_issues}")

## 2. View Discovered Issues

In [ ]:
for issue in result.issues:
    print("=" * 60)
    print(issue.format_for_display())
    print()

## 3. Get Improvement Candidates

These are issues that can be selected for improvement (excludes learning harness).

In [ ]:
candidates = result.get_improvement_candidates()
print(f"Improvement candidates: {len(candidates)}")

for c in candidates:
    print(f"  - [{c.issue_id[:8]}] {c.issue_type.value}")
    print(f"    Location: {c.location.file_path}")
    print(f"    Can improve: {c.is_improvement_candidate()}")

## 4. Create User Selection (Simulated)

This would normally happen through the UI. Here we simulate it.

In [ ]:
if candidates:
    # Select the first candidate
    selected_issue = candidates[0]

    # Create selection (user intent)
    selection = manager.create_selection(
        issue_ids=[selected_issue.issue_id],
        target_file=selected_issue.location.file_path,
        user_notes="Fix this issue via Phase 10 discovery",
    )

    print(f"Selection ID: {selection.selection_id}")
    print(f"Target file: {selection.target_file}")
    print("\nGenerated task description:")
    print("-" * 40)
    print(selection.to_task_description(result.issues))
else:
    print("No improvement candidates found.")

## 5. Preview Integration with Controlled Improvement

This shows what would be passed to Phase 9.G controlled improvement.

In [ ]:
from agents.core.issue_to_improvement import IssueImprovementRequest

if candidates:
    request = IssueImprovementRequest(
        selection=selection,
        issues=result.issues,
        project_root="d:/odibi/examples",
        golden_projects=[],
        require_regression_check=False,
    )

    errors = request.validate()
    if errors:
        print("Validation errors:")
        for e in errors:
            print(f"  - {e}")
    else:
        print("✅ Request is valid!")
        print("\nWould create ControlledImprovementConfig with:")
        print(f"  - target_file: {selection.target_file}")
        print(f"  - task: {request.get_task_description()[:100]}...")

## 6. Direct Detection Test

Test detection on specific content.

In [ ]:
detector = PassiveIssueDetector("d:/odibi")

test_config = """
pipelines:
  - pipeline: test_pipeline
    nodes:
      - name: load_data
        read:
          path: data.csv
      
      - name: dedupe_records
        transformer: deduplicate
        params:
          keys: ["id", "timestamp"]
          # Missing order_by - NON-DETERMINISTIC!
        write:
          path: output
"""

issues = detector.detect_issues_in_file("test.yaml", test_config)
print(f"Detected {len(issues)} issue(s):")
for i in issues:
    print(f"\n{i.format_for_display()}")

## 7. Verify Harness Protection

In [ ]:
from agents.core.controlled_improvement import is_learning_harness_path

test_paths = [
    "d:/odibi/examples/improvement_target.odibi.yaml",
    "d:/odibi/.odibi/learning_harness/scale_join.odibi.yaml",
    ".odibi/learning_harness/test.yaml",
]

for p in test_paths:
    protected = is_learning_harness_path(p)
    emoji = "🔒" if protected else "✅"
    print(f"{emoji} {p}: {'PROTECTED' if protected else 'can improve'}")